In [1]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
import statistics as stat

In [2]:
data = pd.read_csv('scdb-20rfq-15cap-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [3]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

1538

In [4]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

bid_count = 0
bid_ids = []
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'BID':
        bid_ids.append(data_unique_ids.iloc[i]['Tx_id'])
        bid_count += 1   


print(f"#BIDs =  {bid_count}")

#BIDs =  500


In [5]:
df_test = grouped_data.get_group(bid_ids[8])
df_test

,Event,Time,Tx_type,Tx_id,Accept_id,time
3180,received_tx,74,BID,5a936ce76160ca52e6d9e894f2daffe7dee9ad7f76c8c9...,None,1.669789e+09
3181,before_tendermint,86,BID,5a936ce76160ca52e6d9e894f2daffe7dee9ad7f76c8c9...,None,1.669789e+09
3182,check_tx,97,BID,5a936ce76160ca52e6d9e894f2daffe7dee9ad7f76c8c9...,None,1.669789e+09
3183,deliver_tx,134,BID,5a936ce76160ca52e6d9e894f2daffe7dee9ad7f76c8c9...,None,1.669789e+09
3184,end_block,136,BID,5a936ce76160ca52e6d9e894f2daffe7dee9ad7f76c8c9...,None,1.669789e+09
3185,commit_tx,180,BID,5a936ce76160ca52e6d9e894f2daffe7dee9ad7f76c8c9...,None,1.669789e+09


In [6]:
time_all_committed_bids = {}
time_passed_bids = []
validated_time = []
accepted_time = []
committed_time = []

bid_underfit = []
bid_underfit_dict = {}

bid_overfit = []
bid_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "BID" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_bids[tx_id[i]] = actual_time
            time_passed_bids.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            bid_underfit.append(temp_list)
            bid_underfit_dict[tx_id[i]] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_bids[tx_id[i]] = actual_time
            #time_passed_bids.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            
            bid_overfit.append(actual_time)
            bid_overfit_dict[tx_id[i]] = temp_list

In [7]:
df = grouped_data.get_group("3066cad3ab9009c84499c002f3b150864fc0b5af46bd4cb1ab5407ab2396a336")
df[df['Event'] == 'received_tx']['Time'].to_list()[0]

KeyError: '3066cad3ab9009c84499c002f3b150864fc0b5af46bd4cb1ab5407ab2396a336'

In [8]:
df[df['Event'] == 'received_tx']

,Event,Time,Tx_type,Tx_id,Accept_id,time
4614,received_tx,71,BID,ffd75bbe95db4d549184320201fab05b24fb37c834258f...,None,1.669789e+09


In [9]:
bid_overfit_dict

{'19062c86d919000bd541ee703c9b93f4acf2c779b640a5e71329815039bed7cc': [82,
  92,
  103,
  179,
  243,
  244,
  288],
 '26964f1b8a7cb340363a88e7b5ef012727a29bcaec62604483798f03171b2d38': [78,
  88,
  109,
  299,
  359,
  360,
  404],
 '2b377b1db2e62652659a2541b60682ce02009f9505ee33acc5057c0152e47250': [74,
  85,
  107,
  297,
  357,
  359,
  402],
 '4a3475495908f1803be716962f04d8ec20a5a344203e15ab57a2b106e8c1beb8': [75,
  85,
  117,
  336,
  448,
  502,
  510],
 '4ea1b4aa1ee99d32f7211886ae3adb3adc8c570fed1675d41fdf6ebe3e1b8736': [84,
  94,
  120,
  319,
  378,
  380,
  423],
 '605c5375a9483e1405305dbe4ee5f4e9996abd89ead579f71b8de3920f594652': [73,
  84,
  145,
  372,
  500,
  562,
  569],
 '6aefd38abaa991eefef185c676a9686fef7511e31541e3aa91725810146cfcf9': [78,
  89,
  111,
  346,
  402,
  403,
  447],
 '6ccb878794a761e09024e1e1ed914b93221c45caab0ee6d067663dcf8e4639f9': [75,
  85,
  110,
  345,
  404,
  406,
  449],
 'b8ba8323690e0b08f61f3c6efdde0820b3432298cc928b827e7a712b02a2aa24': [73

In [10]:
len(time_passed_bids)

487

In [11]:
len(bid_overfit)

12

In [12]:
stat.mean(time_passed_bids)

101.62217659137578

In [13]:
stat.mean(bid_overfit)

355.4166666666667

In [14]:
# Analysis of CREATE Transaction
bid_mean = "{:.2f}".format(stat.mean(time_passed_bids))
bid_median = stat.median(time_passed_bids)
bid_max = max(time_passed_bids)
bid_min = min(time_passed_bids)
bid_std_dev = "{:.2f}".format(stat.stdev(time_passed_bids))

print(bid_mean)
print(bid_median)
print(bid_max)
print(bid_min)
print(bid_std_dev)
print(f"total bids = {len(time_passed_bids)}")

bid_dict = {
               'min': bid_min,
               'max': bid_max,
               'average': bid_mean,
               'median' : bid_median,
               'standard_deviation' : bid_std_dev,
               '# of txs' : len(time_passed_bids)}

101.62
86
596
74
44.52
total bids = 487


In [15]:
results = pd.DataFrame(bid_dict, index = ['BID'])
results.to_csv('bid_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
BID,74,596,101.62,86,44.52,487
